# Regression kink analysis with `pymc` models

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import causalpy as cp

In [ ]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'
seed = 42
rng = np.random.default_rng(seed)

Regression kink designs should be analysed by a piecewise continuous function. That is:
* We want a function which can capture the data to the left and to the right of the kink point.
* We want a piecewise function with one breakpoint or kink point
* The function should be continuous at the kink point

An example of such a function would be a piecewise continuous polynomial:

$$
\mu = \beta_0 + \beta_1 \cdot x + \beta_2 \cdot x^2 + \beta_3 \cdot (x-k) + \beta_4 \cdot (x-k)^2
$$

We can visualise what these functions look like by plotting a number of them with randomly chosen $\beta$ coefficients, but all with a kink point at $x=0.5$.


In [ ]:
def f(x, beta, kink):
    return (
        beta[0]
        + beta[1] * x
        + beta[2] * x**2
        + beta[3] * (x - kink) * (x >= kink)
        + beta[4] * (x - kink) ** 2 * (x >= kink)
    )


def gradient_change(beta, kink, epsilon=0.01):
    gradient_left = (f(kink, beta, kink) - f(kink - epsilon, beta, kink)) / epsilon
    gradient_right = (f(kink + epsilon, beta, kink) - f(kink, beta, kink)) / epsilon
    gradient_change = gradient_right - gradient_left
    return gradient_change


x = np.linspace(-1, 1, 1000)
kink = 0.5
sigma = 0.05
cols = 5

fig, ax = plt.subplots(2, cols, sharex=True, sharey=True, figsize=(15, 5))

for col in range(cols):
    beta = rng.random(5)
    ax[0, col].plot(x, f(x, beta, kink), lw=3)
    ax[1, col].plot(x, np.gradient(f(x, beta, kink), x), lw=3)
    ax[0, col].set(title=f"Random  {col+1}")
    ax[1, col].set(xlabel="x")

ax[0, 0].set(ylabel="$y = f(x)$")
ax[1, 0].set(ylabel=r"$\frac{dy}{dx}$");

The idea of regression kink analysis is to fit a suitable function to data and to estimate whether there is a change in the gradient of the function at the kink point.

Below we will generate a number of datasets and run through how to conduct the regression kink analysis. We will use a function to generate simulated datasets with the properties we want.

In [ ]:
def generate_data(beta, kink, sigma=0.05, N=50):
    if beta is None:
        beta = rng.random(5)
    x = rng.uniform(-1, 1, N)
    y = f(x, beta, kink) + rng.normal(0, sigma, N)
    df = pd.DataFrame({"x": x, "y": y, "treated": x >= kink})
    return df

## Example 1 - continuous piecewise linear function
In this example we'll stick to a simple continuous piecewise function.

In [ ]:
kink = 0.5
beta = [0, -1, 0, 2, 0]
df = generate_data(beta, kink, sigma=sigma)

fig, ax = plt.subplots()
ax.scatter(df["x"], df["y"], alpha=0.5)
ax.axvline(kink, color="red", linestyle="--")
ax.set(title=f"Change in gradient at kink point: {gradient_change(beta, kink):.2f}");

We can use the regular `cp.pymc_models.LinearRegression` model and enforce the continuous piecewise nature by cleverly specifying a design matrix via the `formula` input.

In this example, setting the formula to `"y ~ 1 + x + I((x-0.5)*treated)"` (where the 0.5 is the kink point) equates to the following model:

$$
\mu = \beta_0 + \beta_1 \cdot x + \beta_3 \cdot (x-k)
$$

In [ ]:
result1 = cp.pymc_experiments.RegressionKink(
    df,
    formula=f"y ~ 1 + x + I((x-{kink})*treated)",
    model=cp.pymc_models.LinearRegression(sample_kwargs={"random_seed": seed}),
    kink_point=kink,
    epsilon=0.1,
)

fig, ax = result1.plot()

If you want to plot the posterior distribution of the inferred gradient change, you can do it as follows.

In [ ]:
ax = az.plot_posterior(result1.gradient_change, ref_val=gradient_change(beta, kink))
ax.set(title="Gradient change");

We know that the correct gradient change is 2, and that we have correctly recovered it as the posterior distribution is centred around 2.

## Example 2 - continuous piecewise polynomial function

Now we'll introduce some nonlinearity into the mix.

In this example, we're going to have a 2nd order polynomial on either side of the kink point. So the model can be defined as:

$$
\mu = \beta_0 + \beta_1 \cdot x + \beta_2 \cdot x^2 + \beta_3 \cdot (x-k) + \beta_4 \cdot (x-k)^2
$$

While it's a bit verbose, we can implement this in a [patsy](https://patsy.readthedocs.io/en/latest/index.html) formula as so:

> `"y ~ 1 + x + np.power(x,2) + I((x-0.5)*treated) + I(np.power((x-0.5), 2)*treated)"` 

where the 0.5 is the kink point.

In [ ]:
kink = 0.5
beta = [0, 0, 1, -1, -5]
df = generate_data(beta, kink, N=200, sigma=sigma)

fig, ax = plt.subplots()
ax.scatter(df["x"], df["y"], alpha=0.5)
ax.axvline(kink, color="red", linestyle="--")
ax.set(title=f"Change in gradient at kink point: {gradient_change(beta, kink):.2f}");

In [ ]:
formula = f"y ~ 1 + x + np.power(x,2) + I((x-{kink})*treated) + I(np.power(x-{kink}, 2)*treated)"

result2 = cp.pymc_experiments.RegressionKink(
    df,
    formula=formula,
    model=cp.pymc_models.LinearRegression(
        sample_kwargs={"random_seed": seed, "tune": 5_000, "target_accept": 0.95}
    ),
    kink_point=kink,
    epsilon=0.01,
)

fig, ax = result2.plot()

We can also evaluate the posterior distribution of the parameters and see how they match up with the true values.

In [ ]:
print(beta, sigma)

In [ ]:
az.plot_forest(result2.idata, var_names="~mu", figsize=(10, 3));

In [ ]:
ax = az.plot_posterior(result2.gradient_change, ref_val=gradient_change(beta, kink))
ax.set(title="Gradient change");

## Example 3 - basis spline model

In [ ]:
result3 = cp.pymc_experiments.RegressionKink(
    df,
    formula=f"y ~ 1 + bs(x, df=3) + bs(I(x-{kink})*treated, df=3)",
    model=cp.pymc_models.LinearRegression(sample_kwargs={"random_seed": seed}),
    kink_point=kink,
)

result3.plot();

In [ ]:
ax = az.plot_posterior(result3.gradient_change, ref_val=gradient_change(beta, kink))
ax.set(title="Gradient change");